# Hot Sauce NLP


In [1]:
import requests
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib

In [2]:
def fetch_page(page):
    print(f"scraping page {page + 1}")
    response = requests.get(
        f'https://hotsaucefever.com/hotsauce/',
        headers={"Accept-Language":"en-US"}
    )
    soup = BeautifulSoup(response.content, "html.parser")
    return soup

In [3]:

def add_sauces_to_dict(soup, dict):
    for sauce_html in soup.find_all(class_='detail'):
        dict['Hot Sauce'].append(sauce_html.find('h2').find('a').string)
        dict['Rating'].append(sauce_html.find('h3').find('span').string)
        dict['Makers'].append(
            sauce_html.find(class_="list-txt").get_text())

In [4]:
def create_sauces_df(max_page):
    sauces_dict = {'Hot Sauce': [], 'Rating': [], 'Makers': []}
    for page in range(max_page):
        soup = fetch_page(page)
        add_sauces_to_dict(soup, sauces_dict)
    return pd.DataFrame.from_dict(sauces_dict)

In [31]:
sauces_df = create_sauces_df(25)
sauces_df.shape

scraping page 1
scraping page 2
scraping page 3
scraping page 4
scraping page 5
scraping page 6
scraping page 7
scraping page 8
scraping page 9
scraping page 10
scraping page 11
scraping page 12
scraping page 13
scraping page 14
scraping page 15
scraping page 16
scraping page 17
scraping page 18
scraping page 19
scraping page 20
scraping page 21
scraping page 22
scraping page 23
scraping page 24
scraping page 25


(625, 3)

In [32]:
sauces_df


,Hot Sauce,Rating,Makers
0,Whitehouse Station Sauce Company - Jalapeño,N/A,"Maker: Whitehouse Station Sauce Company, White..."
1,Jersey Barnfire - Black Garlic,None,"Maker: Jersey Barnfire, Sparta, NJ, United Sta..."
2,Roofeeo Landis - Jah Mama Sauce,N/A,"Maker: Roofeeo Landis, Los Angeles , CA, Unite..."
3,Ric Orlando's Best - Purple Haze Double Dose,N/A,"Maker: Ric Orlando's Best, West Hurley, NY, Un..."
4,Hell's Kitchen Hot Sauce - Pepper Pastry,N/A,"Maker: Hell's Kitchen Hot Sauce, New York, NY,..."
...,...,...,...
620,Iggy's - Fermented Habanero Hot Sauce,N/A,"Maker: Iggy's Alive & Cultured, Bainbridge Isl..."
621,Bullwhip - Kelp Hot Sauce,N/A,"Maker: Barnacle Foods, Juneau, AK, United Stat..."
622,oo'mämē - Moroccan Chile Crisp,N/A,"Maker: Fifth Taste Foods, LLC, St Louis, MO, U..."
623,oo'mämē - Indian Chile Crisp,N/A,"Maker: Fifth Taste Foods, LLC, St Louis, MO, U..."


In [33]:
# new data frame with split value columns
new = sauces_df["Makers"].str.split("Maker:", n =1, expand = True)

# making separate first name column from new data frame
#books_df["Makers"]= new[0]
  
# making separate last name column from new data frame
#books_df["Peppers"]= new[1]

#new1 = new[1].str.split("Pepper(s):", n = 1, expand = True)

In [34]:
new


,0,1
0,,"Whitehouse Station Sauce Company, Whitehouse ..."
1,,"Jersey Barnfire, Sparta, NJ, United States\n ..."
2,,"Roofeeo Landis, Los Angeles , CA, United Stat..."
3,,"Ric Orlando's Best, West Hurley, NY, United S..."
4,,"Hell's Kitchen Hot Sauce, New York, NY, Unite..."
...,...,...
620,,"Iggy's Alive & Cultured, Bainbridge Island, W..."
621,,"Barnacle Foods, Juneau, AK, United States\n ..."
622,,"Fifth Taste Foods, LLC, St Louis, MO, United ..."
623,,"Fifth Taste Foods, LLC, St Louis, MO, United ..."


In [35]:
new1 = new[1].str.split(":", expand = True)


# 

In [36]:
cols = [0, 1, 2]
new1[cols] = new1[cols].apply(lambda x: x.str.strip('%\n'))
new1[cols] = new1[cols].apply(lambda x: x.str.strip(' \n%'))
new1[cols] = new1[cols].apply(lambda x: x.str.strip('%\n'))
new1[cols] = new1[cols].apply(lambda x: x.str.strip('%\nPepper(s)'))
new1[cols] = new1[cols].apply(lambda x: x.str.strip('\n'))
new1[cols] = new1[cols].apply(lambda x: x.str.strip('SHU'))

In [37]:
new1

,0,1,2
0,"Whitehouse Station Sauce Company, Whitehouse S...",Jalapeño,None
1,"Jersey Barnfire, Sparta, NJ, United States\n ...","abanero, Jalapeño",None
2,"Roofeeo Landis, Los Angeles , CA, United State...","cotch Bonnet, Fresno",None
3,"Ric Orlando's Best, West Hurley, NY, United St...","abanero, Carolina Rea",None
4,"ell's Kitchen Hot Sauce, New York, NY, United ...","abanero, Fatalii",None
...,...,...,...
620,"Iggy's Alive & Cultured, Bainbridge Island, WA...",abanero,None
621,"Barnacle Foods, Juneau, AK, United States\n ...",i Peri,None
622,"Fifth Taste Foods, LLC, St Louis, MO, United Stat",None,None
623,"Fifth Taste Foods, LLC, St Louis, MO, United Stat",None,None


In [38]:
result = pd.concat([sauces_df, new1], axis=1, ignore_index=True)

In [39]:
result.drop(result.columns[[2]], axis=1, inplace=True)

In [40]:
result

,0,1,3,4,5
0,Whitehouse Station Sauce Company - Jalapeño,N/A,"Whitehouse Station Sauce Company, Whitehouse S...",Jalapeño,None
1,Jersey Barnfire - Black Garlic,None,"Jersey Barnfire, Sparta, NJ, United States\n ...","abanero, Jalapeño",None
2,Roofeeo Landis - Jah Mama Sauce,N/A,"Roofeeo Landis, Los Angeles , CA, United State...","cotch Bonnet, Fresno",None
3,Ric Orlando's Best - Purple Haze Double Dose,N/A,"Ric Orlando's Best, West Hurley, NY, United St...","abanero, Carolina Rea",None
4,Hell's Kitchen Hot Sauce - Pepper Pastry,N/A,"ell's Kitchen Hot Sauce, New York, NY, United ...","abanero, Fatalii",None
...,...,...,...,...,...
620,Iggy's - Fermented Habanero Hot Sauce,N/A,"Iggy's Alive & Cultured, Bainbridge Island, WA...",abanero,None
621,Bullwhip - Kelp Hot Sauce,N/A,"Barnacle Foods, Juneau, AK, United States\n ...",i Peri,None
622,oo'mämē - Moroccan Chile Crisp,N/A,"Fifth Taste Foods, LLC, St Louis, MO, United Stat",None,None
623,oo'mämē - Indian Chile Crisp,N/A,"Fifth Taste Foods, LLC, St Louis, MO, United Stat",None,None


In [41]:
new2 = result[0].str.split(" - ", expand = True)

In [42]:
new2

,0,1
0,Whitehouse Station Sauce Company,Jalapeño
1,Jersey Barnfire,Black Garlic
2,Roofeeo Landis,Jah Mama Sauce
3,Ric Orlando's Best,Purple Haze Double Dose
4,Hell's Kitchen Hot Sauce,Pepper Pastry
...,...,...
620,Iggy's,Fermented Habanero Hot Sauce
621,Bullwhip,Kelp Hot Sauce
622,oo'mämē,Moroccan Chile Crisp
623,oo'mämē,Indian Chile Crisp


In [43]:
result1 = pd.concat([result, new2], axis=1, ignore_index=True)

In [44]:
result1

,0,1,2,3,4,5,6
0,Whitehouse Station Sauce Company - Jalapeño,N/A,"Whitehouse Station Sauce Company, Whitehouse S...",Jalapeño,None,Whitehouse Station Sauce Company,Jalapeño
1,Jersey Barnfire - Black Garlic,None,"Jersey Barnfire, Sparta, NJ, United States\n ...","abanero, Jalapeño",None,Jersey Barnfire,Black Garlic
2,Roofeeo Landis - Jah Mama Sauce,N/A,"Roofeeo Landis, Los Angeles , CA, United State...","cotch Bonnet, Fresno",None,Roofeeo Landis,Jah Mama Sauce
3,Ric Orlando's Best - Purple Haze Double Dose,N/A,"Ric Orlando's Best, West Hurley, NY, United St...","abanero, Carolina Rea",None,Ric Orlando's Best,Purple Haze Double Dose
4,Hell's Kitchen Hot Sauce - Pepper Pastry,N/A,"ell's Kitchen Hot Sauce, New York, NY, United ...","abanero, Fatalii",None,Hell's Kitchen Hot Sauce,Pepper Pastry
...,...,...,...,...,...,...,...
620,Iggy's - Fermented Habanero Hot Sauce,N/A,"Iggy's Alive & Cultured, Bainbridge Island, WA...",abanero,None,Iggy's,Fermented Habanero Hot Sauce
621,Bullwhip - Kelp Hot Sauce,N/A,"Barnacle Foods, Juneau, AK, United States\n ...",i Peri,None,Bullwhip,Kelp Hot Sauce
622,oo'mämē - Moroccan Chile Crisp,N/A,"Fifth Taste Foods, LLC, St Louis, MO, United Stat",None,None,oo'mämē,Moroccan Chile Crisp
623,oo'mämē - Indian Chile Crisp,N/A,"Fifth Taste Foods, LLC, St Louis, MO, United Stat",None,None,oo'mämē,Indian Chile Crisp


In [45]:
result1.drop(result1.columns[[0]], axis=1, inplace=True)

In [46]:
result1.columns = ['Rating', 'Address', 'Peppers', 'Scoville', 'Company', 'Sauce']

In [47]:
result1

,Rating,Address,Peppers,Scoville,Company,Sauce
0,N/A,"Whitehouse Station Sauce Company, Whitehouse S...",Jalapeño,None,Whitehouse Station Sauce Company,Jalapeño
1,None,"Jersey Barnfire, Sparta, NJ, United States\n ...","abanero, Jalapeño",None,Jersey Barnfire,Black Garlic
2,N/A,"Roofeeo Landis, Los Angeles , CA, United State...","cotch Bonnet, Fresno",None,Roofeeo Landis,Jah Mama Sauce
3,N/A,"Ric Orlando's Best, West Hurley, NY, United St...","abanero, Carolina Rea",None,Ric Orlando's Best,Purple Haze Double Dose
4,N/A,"ell's Kitchen Hot Sauce, New York, NY, United ...","abanero, Fatalii",None,Hell's Kitchen Hot Sauce,Pepper Pastry
...,...,...,...,...,...,...
620,N/A,"Iggy's Alive & Cultured, Bainbridge Island, WA...",abanero,None,Iggy's,Fermented Habanero Hot Sauce
621,N/A,"Barnacle Foods, Juneau, AK, United States\n ...",i Peri,None,Bullwhip,Kelp Hot Sauce
622,N/A,"Fifth Taste Foods, LLC, St Louis, MO, United Stat",None,None,oo'mämē,Moroccan Chile Crisp
623,N/A,"Fifth Taste Foods, LLC, St Louis, MO, United Stat",None,None,oo'mämē,Indian Chile Crisp


In [48]:
result1 = result1[['Sauce', 'Company', 'Rating', 'Scoville', 'Address']]

In [49]:
result1

,Sauce,Company,Rating,Scoville,Address
0,Jalapeño,Whitehouse Station Sauce Company,N/A,None,"Whitehouse Station Sauce Company, Whitehouse S..."
1,Black Garlic,Jersey Barnfire,None,None,"Jersey Barnfire, Sparta, NJ, United States\n ..."
2,Jah Mama Sauce,Roofeeo Landis,N/A,None,"Roofeeo Landis, Los Angeles , CA, United State..."
3,Purple Haze Double Dose,Ric Orlando's Best,N/A,None,"Ric Orlando's Best, West Hurley, NY, United St..."
4,Pepper Pastry,Hell's Kitchen Hot Sauce,N/A,None,"ell's Kitchen Hot Sauce, New York, NY, United ..."
...,...,...,...,...,...
620,Fermented Habanero Hot Sauce,Iggy's,N/A,None,"Iggy's Alive & Cultured, Bainbridge Island, WA..."
621,Kelp Hot Sauce,Bullwhip,N/A,None,"Barnacle Foods, Juneau, AK, United States\n ..."
622,Moroccan Chile Crisp,oo'mämē,N/A,None,"Fifth Taste Foods, LLC, St Louis, MO, United Stat"
623,Indian Chile Crisp,oo'mämē,N/A,None,"Fifth Taste Foods, LLC, St Louis, MO, United Stat"
